# DICOM Processing Notebook

This notebook processes DICOM files and sends the results to the server endpoint.
It ensures consistent image processing by running in the same environment as your original working code.


In [ ]:
import os, sys, numpy as np, pydicom
from threading import Event
from matplotlib import pyplot as plt
from PIL import Image
import requests
import json
from pathlib import Path
import base64
from io import BytesIO

# Cooperative cancellation support
cancel_event: Event = Event()


In [ ]:
# ---- load frames from a DICOM (exact match to your reference) ----
def load_frames(ds):
    arr = ds.pixel_array

    if arr.ndim == 2:                      # single frame gray
        frames = [arr]

    elif arr.ndim == 3:
        if arr.shape[-1] in (3, 4):        # (H, W, 3/4) -> one RGB frame
            frames = [arr[..., :3]]
        else:                               # (N, H, W) stack
            frames = [arr[i] for i in range(arr.shape[0])]

    elif arr.ndim == 4:                     # (N, H, W, 3/4)
        frames = [arr[i, ..., :3] for i in range(arr.shape[0])]

    else:
        frames = []

    return frames


In [ ]:
# ---- save frames as PNGs (exact match to your reference) ----
def save_pngs(frames, base, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    written = []
    for i, f in enumerate(frames, 1):
        if cancel_event.is_set():
            break
            
        if f.ndim == 3:  # RGB to grayscale
            f = np.mean(f, axis=-1)

        out_name = f"{base}_frame_{i:03d}.png"
        out_path = os.path.join(out_dir, out_name)
        plt.imsave(out_path, f, cmap="gray", vmin=f.min(), vmax=f.max())
        written.append(out_path)
    return written


In [ ]:
# ---- crop PNGs with the exact crop box ----
def crop_pngs(files, out_dir, top=65, left=66, right=150):
    os.makedirs(out_dir, exist_ok=True)
    written = []
    for fp in files:
        if cancel_event.is_set():
            break
        img = Image.open(fp)
        w, h = img.size
        # crop(left, top, right, bottom) -> right is x-coordinate, so use w - right
        cropped = img.crop((left, top, w - right, h))
        out_path = os.path.join(out_dir, os.path.basename(fp))
        cropped.save(out_path)
        written.append(out_path)
    return written


In [ ]:
def process_single_dcm(dcm_path, out_raw="Processed_PNGs", out_cropped="Processed_PNGs_cropped"):
    ds = pydicom.dcmread(dcm_path)
    frames = load_frames(ds)
    if not frames:
        return [], []

    # Use the file's stem as the base name (no special parsing)
    base = os.path.splitext(os.path.basename(dcm_path))[0]
    if cancel_event.is_set():
        return [], []
    raw_files = save_pngs(frames, base, out_raw)
    if cancel_event.is_set():
        return raw_files, []
    cropped_files = crop_pngs(raw_files, out_cropped, top=65, left=66, right=150)
    return raw_files, cropped_files


In [ ]:
def send_images_to_endpoint(cropped_files, server_url="http://localhost:8000"):
    """
    Send processed images to the server endpoint
    """
    results = []
    
    for img_path in cropped_files:
        try:
            # Read the image file
            with open(img_path, 'rb') as img_file:
                img_data = img_file.read()
            
            # Convert to base64 for sending
            img_base64 = base64.b64encode(img_data).decode('utf-8')
            
            # Prepare the payload
            payload = {
                "image": img_base64,
                "filename": os.path.basename(img_path)
            }
            
            # Send to endpoint (you'll need to create this endpoint)
            response = requests.post(f"{server_url}/process_image", 
                                   json=payload,
                                   headers={"Content-Type": "application/json"})
            
            if response.status_code == 200:
                results.append({
                    "file": img_path,
                    "status": "success",
                    "response": response.json()
                })
                print(f"✅ Successfully processed {os.path.basename(img_path)}")
            else:
                results.append({
                    "file": img_path,
                    "status": "error",
                    "error": response.text
                })
                print(f"❌ Error processing {os.path.basename(img_path)}: {response.text}")
                
        except Exception as e:
            results.append({
                "file": img_path,
                "status": "error",
                "error": str(e)
            })
            print(f"❌ Exception processing {os.path.basename(img_path)}: {e}")
    
    return results


## Usage Example

Run the cells below to process a DICOM file and send results to your endpoint:


In [ ]:
# Configuration
DCM_PATH = "/path/to/your/file.dcm"  # Update this path
SERVER_URL = "http://localhost:8000"  # Update if your server runs on different port
OUT_RAW = "jupyter_processed_raw"
OUT_CROPPED = "jupyter_processed_cropped"

# Check if DICOM file exists
if not Path(DCM_PATH).exists():
    print(f"❌ DICOM file not found: {DCM_PATH}")
    print("Please update the DCM_PATH variable with the correct path to your DICOM file.")
else:
    print(f"✅ Found DICOM file: {DCM_PATH}")


In [ ]:
# Process the DICOM file
if Path(DCM_PATH).exists():
    try:
        print("🔄 Processing DICOM file...")
        raw_files, cropped_files = process_single_dcm(DCM_PATH, OUT_RAW, OUT_CROPPED)
        
        print(f"✅ Wrote {len(raw_files)} raw PNG frames to {OUT_RAW}")
        print(f"✅ Wrote {len(cropped_files)} cropped PNGs to {OUT_CROPPED}")
        
        # Display first cropped image
        if cropped_files:
            img = Image.open(cropped_files[0])
            plt.figure(figsize=(10, 8))
            plt.imshow(img, cmap="gray")
            plt.title(f"First cropped image: {os.path.basename(cropped_files[0])}")
            plt.axis("off")
            plt.show()
            
    except Exception as e:
        print(f"❌ Failed to process DICOM: {e}")


In [ ]:
# Send processed images to endpoint
if 'cropped_files' in locals() and cropped_files:
    print("🔄 Sending images to server endpoint...")
    results = send_images_to_endpoint(cropped_files, SERVER_URL)
    
    # Summary
    successful = sum(1 for r in results if r['status'] == 'success')
    failed = sum(1 for r in results if r['status'] == 'error')
    
    print(f"\n📊 Summary:")
    print(f"✅ Successfully processed: {successful} images")
    print(f"❌ Failed: {failed} images")
    
    if failed > 0:
        print("\n❌ Failed files:")
        for r in results:
            if r['status'] == 'error':
                print(f"  - {os.path.basename(r['file'])}: {r['error']}")
else:
    print("⚠️ No cropped files to send. Please run the processing cell first.")
